In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import json 
import matplotlib
import matplotlib.pyplot as plt


from sklearn.metrics import jaccard_score
from scipy.spatial.distance import directed_hausdorff

import cv2
import tifffile
import seaborn as sns
from itertools import chain
from PIL import Image
from PIL import ImageDraw

import skimage
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import scipy.stats as stats

In [ ]:
#Read Metrics file for the team
BASE_PATH = os.getcwd()+'/'
INPUT_PATH = BASE_PATH
print(INPUT_PATH)

In [3]:
def get_stats(df_metrics_team1, metric, case, per_organ, team_num, n_case):
    if case == "best":
        if per_organ is True:
            indices_to_drop = df_metrics_team1.groupby('tissue_name')[metric].nlargest(n_case).index.get_level_values(1)
            team1_stats = df_metrics_team1.drop(indices_to_drop, inplace=False)

        elif per_organ is False:
            df_metrics_team1_sorted = df_metrics_team1.sort_values(metric, ascending=False)
            df_metrics_team1_sorted = df_metrics_team1_sorted.reset_index(drop=True)
            team1_stats = df_metrics_team1_sorted.drop(df_metrics_team1_sorted[df_metrics_team1_sorted['id'] == df_metrics_team1_sorted.id[0]].index)

    elif case == "worst":
        if per_organ is True:
            indices_to_drop = df_metrics_team1.groupby('tissue_name')[metric].nsmallest(n_case).index.get_level_values(1)
            team1_stats = df_metrics_team1.drop(indices_to_drop, inplace=False)

        elif per_organ is False:
            df_metrics_team1_sorted = df_metrics_team1.sort_values(metric, ascending=True)
            df_metrics_team1_sorted = df_metrics_team1_sorted.reset_index(drop=True)
            team1_stats = df_metrics_team1_sorted.drop(df_metrics_team1_sorted[df_metrics_team1_sorted['id'] == df_metrics_team1_sorted.id[0]].index)
    mean_val = team1_stats[metric].mean()
    # print(f"Team {team_num}: {mean_val}")
    return mean_val
    

In [4]:
column_names = ['TeamName', 'dice_score', 'iou_score']
top50_ranking_dropped1 = pd.DataFrame(columns=column_names)
top50_ranking_dropped3 = pd.DataFrame(columns=column_names)

In [5]:
for i in range(1,51):
    # metric = "iou_score"
    team = f"team_{i}"
    if i == 1 or i == 2 or i == 3:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')
    else:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/other_top50_teams/{team}/metrics.csv')

    metric = "dice_score"
    case = "worst"
    per_organ = True
    mean_val_dice = get_stats(df_metrics, metric, case, per_organ, i, 1)

    metric = "iou_score"
    case = "worst"
    per_organ = True
    mean_val_iou = get_stats(df_metrics, metric, case, per_organ, i, 1)

    new_row = {'TeamName' : team, 'dice_score' : mean_val_dice, 'iou_score' : mean_val_iou}

    top50_ranking_dropped1 = top50_ranking_dropped1.append(new_row, ignore_index=True)

    # print(f"Team completed: {team}")
    
# print("Metric computation complete")

/tmp/ipykernel_4043488/2637394250.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped1 = top50_ranking_dropped1.append(new_row, ignore_index=True)
/tmp/ipykernel_4043488/2637394250.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped1 = top50_ranking_dropped1.append(new_row, ignore_index=True)
/tmp/ipykernel_4043488/2637394250.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped1 = top50_ranking_dropped1.append(new_row, ignore_index=True)
/tmp/ipykernel_4043488/2637394250.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped1 = top50_ranking_dropped1

In [6]:
display(top50_ranking_dropped1)

,TeamName,dice_score,iou_score
0,team_1,0.842140,0.745262
1,team_2,0.841845,0.744692
2,team_3,0.841300,0.742327
3,team_4,0.832170,0.732461
4,team_5,0.833201,0.734884
5,team_6,0.831330,0.734160
6,team_7,0.831345,0.732670
7,team_8,0.831908,0.731414
8,team_9,0.830406,0.731740
9,team_10,0.830115,0.730888


In [8]:
for j in range(1,51):
    # metric = "iou_score"
    team = f"team_{j}"
    if j == 1 or j == 2 or j == 3:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')
    else:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/other_top50_teams/{team}/metrics.csv')

    metric = "dice_score"
    case = "worst"
    per_organ = True
    mean_val_dice = get_stats(df_metrics, metric, case, per_organ, j, 3)

    metric = "iou_score"
    case = "worst"
    per_organ = True
    mean_val_iou = get_stats(df_metrics, metric, case, per_organ, j, 3)

    new_row = {'TeamName' : team, 'dice_score' : mean_val_dice, 'iou_score' : mean_val_iou}

    top50_ranking_dropped3 = top50_ranking_dropped3.append(new_row, ignore_index=True)

/tmp/ipykernel_4043488/4006824951.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped3 = top50_ranking_dropped3.append(new_row, ignore_index=True)
/tmp/ipykernel_4043488/4006824951.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped3 = top50_ranking_dropped3.append(new_row, ignore_index=True)
/tmp/ipykernel_4043488/4006824951.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped3 = top50_ranking_dropped3.append(new_row, ignore_index=True)
/tmp/ipykernel_4043488/4006824951.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top50_ranking_dropped3 = top50_ranking_dropped3

In [9]:
display(top50_ranking_dropped3)

,TeamName,dice_score,iou_score
0,team_1,0.850595,0.755482
1,team_2,0.850365,0.754829
2,team_3,0.850870,0.753838
3,team_4,0.843226,0.744912
4,team_5,0.842257,0.745573
5,team_6,0.839558,0.743950
6,team_7,0.839129,0.742010
7,team_8,0.841222,0.742264
8,team_9,0.839384,0.742028
9,team_10,0.838788,0.741288


# Top 50 teams analysis

In [3]:
column_names = ['TeamName', 'dice_score', 'iou_score', 'hd_score', 'hd95_score']
df_all_metrics_top50 = pd.DataFrame(columns=column_names)
df_all_metrics_top50

,TeamName,dice_score,iou_score


In [4]:
for i in range(1,51):
    # metric = "iou_score"
    team = f"team_{i}"
    if i == 1 or i == 2 or i == 3:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/{team}/metrics.csv')
    else:
        df_metrics = pd.read_csv(INPUT_PATH+f'winning-submissions/other_top50_teams/{team}/metrics.csv')
    # sorted_df = df_metrics.sort_values(by='PrivateScore', ascending=False)
    dice_mean = df_metrics['dice_score'].mean()
    iou_mean = df_metrics['iou_score'].mean()
    hd_mean = df_metrics['hd_score'].mean()
    hd95_mean = df_metrics['hd95_score'].mean()

    new_row = {'TeamName' : team, 'dice_score' : dice_mean, 'iou_score' : iou_mean, 'hd_score' : hd_mean, 'hd95_score' : hd95_mean}

    df_all_metrics_top50 = df_all_metrics_top50.append(new_row, ignore_index=True)

# df_all_metrics_top50.to_csv(INPUT_PATH+f'winning-submissions/top50_team_metrics.csv',index=None)


/tmp/ipykernel_441419/2547728947.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_metrics_top50 = df_all_metrics_top50.append(new_row, ignore_index=True)
/tmp/ipykernel_441419/2547728947.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_metrics_top50 = df_all_metrics_top50.append(new_row, ignore_index=True)
/tmp/ipykernel_441419/2547728947.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_metrics_top50 = df_all_metrics_top50.append(new_row, ignore_index=True)
/tmp/ipykernel_441419/2547728947.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all_metrics_top50 = df_all_metrics_top50.append(new_row, ign

In [18]:
metric = "iou_score" 
top50_ranking_dropped1_sorted = top50_ranking_dropped1.sort_values(metric, ascending=False)
top50_ranking_dropped1_sorted.reset_index(drop=True, inplace=True)
top50_ranking_dropped3_sorted = top50_ranking_dropped3.sort_values(metric, ascending=False)
top50_ranking_dropped3_sorted.reset_index(drop=True, inplace=True)

In [25]:
df_all_metrics_top50_sorted_iou = df_all_metrics_top50.sort_values("iou_score", ascending=False)
df_all_metrics_top50_sorted_iou.reset_index(drop=True, inplace=True)

In [26]:
display(df_all_metrics_top50_sorted_iou, top50_ranking_dropped1_sorted, top50_ranking_dropped3_sorted)

,TeamName,dice_score,iou_score,hd_score,hd95_score
0,team_1,0.835616,0.738462,201.991753,380.928247
1,team_2,0.833929,0.736247,261.016578,392.336002
2,team_3,0.832661,0.733348,268.145931,406.362649
3,team_6,0.825355,0.727711,212.415925,368.317201
4,team_5,0.825953,0.727103,186.205165,363.487386
5,team_7,0.825162,0.725917,289.557443,386.677818
6,team_4,0.824744,0.724394,202.038490,389.344642
7,team_9,0.823342,0.724083,178.454873,369.256189
8,team_8,0.824804,0.724025,240.503372,399.504098
9,team_10,0.822665,0.722829,249.206046,385.919631


,TeamName,dice_score,iou_score
0,team_1,0.842140,0.745262
1,team_2,0.841845,0.744692
2,team_3,0.841300,0.742327
3,team_5,0.833201,0.734884
4,team_6,0.831330,0.734160
5,team_7,0.831345,0.732670
6,team_4,0.832170,0.732461
7,team_9,0.830406,0.731740
8,team_8,0.831908,0.731414
9,team_10,0.830115,0.730888


,TeamName,dice_score,iou_score
0,team_1,0.850595,0.755482
1,team_2,0.850365,0.754829
2,team_3,0.850870,0.753838
3,team_5,0.842257,0.745573
4,team_4,0.843226,0.744912
5,team_6,0.839558,0.743950
6,team_8,0.841222,0.742264
7,team_9,0.839384,0.742028
8,team_7,0.839129,0.742010
9,team_10,0.838788,0.741288


In [ ]:
# df_all_metrics_top50_sorted_dice = df_all_metrics_top50.sort_values(by='dice_score', ascending=False)
# df_all_metrics_top50_sorted_iou = df_all_metrics_top50.sort_values(by='iou_score', ascending=False)
# df_all_metrics_top50_sorted_hd = df_all_metrics_top50.sort_values(by='hd_score', ascending=False)
# df_all_metrics_top50_sorted_hd95 = df_all_metrics_top50.sort_values(by='hd95_score', ascending=False)

In [30]:
arg1 = df_all_metrics_top50['TeamName']
arg2 = df_all_metrics_top50_sorted_iou['TeamName']
tau, p_value = stats.kendalltau(arg1, arg2)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.746938775510204
p-value: 1.9505758390177287e-14


In [31]:
arg1 = df_all_metrics_top50['TeamName']
arg2 = top50_ranking_dropped1_sorted['TeamName']
tau, p_value = stats.kendalltau(arg1, arg2)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.7502040816326531
p-value: 1.502674554357366e-14


In [32]:
arg1 = df_all_metrics_top50['TeamName']
arg2 = top50_ranking_dropped3_sorted['TeamName']
tau, p_value = stats.kendalltau(arg1, arg2)

# Print the calculated tau and p-value
print("Kendall's tau:", tau)
print("p-value:", p_value)

Kendall's tau: 0.7306122448979592
p-value: 7.070885243100395e-14
